In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import zipfile
import time
import copy

In [4]:
import os
import glob
import random
import numpy as np
from collections import Counter
from PIL import Image
from tqdm import tqdm

# --- 설정 ---
DATA_ROOT = 'C:/Users/user/Downloads/Dataset/Dataset' # 데이터가 있는 루트 폴더
SAMPLE_SIZE_FOR_SIZE_CHECK = 1000 # 크기 분석을 위해 몇 장을 샘플링할지

# 이미지로 간주할 확장자들
IMAGE_EXTS = {'.jpg'}

def analyze_folder(folder_path):
    """
    폴더 내의 파일 개수, 확장자 분포, 그리고 이미지 크기 통계를 반환합니다.
    """
    if not os.path.exists(folder_path):
        return None

    total_files = 0
    extension_counts = Counter()
    all_image_paths = []

    # 1. 파일 스캔 (os.walk로 구석구석 찾기)
    print(f"   📂 스캔 중... '{os.path.basename(folder_path)}'")
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            total_files += 1
            ext = os.path.splitext(file)[1].lower()
            extension_counts[ext] += 1
            
            if ext in IMAGE_EXTS:
                all_image_paths.append(os.path.join(root, file))
    
    # 2. 이미지 크기 분석 (샘플링)
    width_stats = {'min': 0, 'max': 0, 'mean': 0}
    height_stats = {'min': 0, 'max': 0, 'mean': 0}
    
    if all_image_paths:
        # 샘플링 (전체 개수가 샘플 수보다 적으면 전체 사용)
        if len(all_image_paths) > SAMPLE_SIZE_FOR_SIZE_CHECK:
            sampled_paths = random.sample(all_image_paths, SAMPLE_SIZE_FOR_SIZE_CHECK)
        else:
            sampled_paths = all_image_paths
            
        widths = []
        heights = []
        
        for img_path in tqdm(sampled_paths, desc=f"   📏 크기 측정 중 ({len(sampled_paths)}장)", leave=False):
            try:
                with Image.open(img_path) as img:
                    w, h = img.size
                    widths.append(w)
                    heights.append(h)
            except Exception:
                pass # 깨진 이미지는 무시
        
        if widths:
            widths = np.array(widths)
            heights = np.array(heights)
            width_stats = {'min': widths.min(), 'max': widths.max(), 'mean': widths.mean()}
            height_stats = {'min': heights.min(), 'max': heights.max(), 'mean': heights.mean()}

    return {
        'total': total_files,
        'exts': extension_counts,
        'w_stats': width_stats,
        'h_stats': height_stats,
        'img_count': len(all_image_paths) # 실제 이미지 파일 수
    }

def main():
    print(f"=== 데이터셋 정밀 분석 (개수 + 크기) ===")
    print(f"대상 경로: {os.path.abspath(DATA_ROOT)}\n")

    if not os.path.exists(DATA_ROOT):
        print(f"❌ 오류: '{DATA_ROOT}' 폴더가 없습니다.")
        return

    try:
        subfolders = [f for f in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, f))]
        subfolders.sort()
    except Exception as e:
        print(f"오류: {e}")
        return

    if not subfolders:
        print("❌ 하위 폴더가 없습니다.")
        return

    print(f"{'폴더명':<12} | {'파일 수':<9} | {'이미지 수':<9} | {'평균 크기 (WxH)':<18} | {'확장자 분포'}")
    print("-" * 95)

    total_images_sum = 0

    for folder in subfolders:
        folder_path = os.path.join(DATA_ROOT, folder)
        result = analyze_folder(folder_path)
        
        if result is None:
            print(f"{folder:<12} | {'경로 없음':<9} |")
            continue
            
        # 결과 포매팅
        count_str = f"{result['total']:,}"
        img_count_str = f"{result['img_count']:,}"
        
        w_mean = result['w_stats']['mean']
        h_mean = result['h_stats']['mean']
        size_str = f"{w_mean:.0f}x{h_mean:.0f}" if w_mean > 0 else "N/A"
        
        # 주요 확장자만 표시 (상위 3개)
        top_exts = result['exts'].most_common(3)
        ext_str = ", ".join([f"{k} {v}" for k, v in top_exts])
        
        print(f"{folder:<12} | {count_str:<9} | {img_count_str:<9} | {size_str:<18} | {ext_str}")
        
        total_images_sum += result['img_count']

    print("-" * 95)
    print(f"총 이미지 파일 합계: {total_images_sum:,} 장")
    print("※ 평균 크기는 폴더별 최대 1,000장 샘플링 기준입니다.")

if __name__ == "__main__":
    main()

=== 데이터셋 정밀 분석 (개수 + 크기) ===
대상 경로: C:\Users\user\Downloads\Dataset\Dataset

폴더명          | 파일 수      | 이미지 수     | 평균 크기 (WxH)        | 확장자 분포
-----------------------------------------------------------------------------------------------
   📂 스캔 중... 'Test'


Test         | 10,906    | 10,905    | 256x256            | .jpg 10905,  1
   📂 스캔 중... 'Train'


Train        | 140,002   | 140,002   | 256x256            | .jpg 140002
   📂 스캔 중... 'Validation'


Validation   | 39,428    | 39,428    | 256x256            | .jpg 39428
-----------------------------------------------------------------------------------------------
총 이미지 파일 합계: 190,335 장
※ 평균 크기는 폴더별 최대 1,000장 샘플링 기준입니다.


In [14]:
IMG_SIZE = 224
EPOCHS = 30
BATCH_SIZE = 32
NUM_SAMPLES = 20000
LEARNING_RATE = 1e-3
PATIENCE = 5

In [6]:
class ResizeWithPad:
    def __init__(self, target_size):
        self.target_size = target_size

    def __call__(self, img):
        w, h = img.size

        scale = self.target_size / max(w, h)

        new_w = int(w * scale)
        new_h = int(h * scale)

        # 비율 유지하며 리사이즈
        resized_img = img.resize((new_w, new_h), Image.LANCZOS)

        # 검은색 캔버스 생성
        canvas = Image.new("RGB", (self.target_size, self.target_size), (0, 0, 0))

        # 캔버스 중앙에 리사이즈된 이미지 배치
        pad_x = (self.target_size - new_w) // 2
        pad_y = (self.target_size - new_h) // 2

        canvas.paste(resized_img, (pad_x, pad_y))

        return canvas

In [7]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
class RAMDataset(Dataset):
    def __init__(self, images_uint8, labels, transform=None):
        self.images = images_uint8 # (N, C, H, W) uint8 tensor
        self.labels = labels
        self.transform = transform
        # 정규화용 transform (여기서 float변환 및 정규화 수행)
        self.normalize = transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        # 1. uint8 텐서 가져오기
        img = self.images[idx]

        # 2. float32로 변환 및 0~1 스케일링 (매우 빠름)
        img = img.float() / 255.0

        # 3. 정규화 적용
        img = self.normalize(img)

        # 4. 추가 증강(Augmentation)이 있다면 여기서 적용 가능
        if self.transform:
            img = self.transform(img)

        label = self.labels[idx]
        return img, label

def load_images_to_ram_uint8(paths):
    """
    이미지를 읽어서 리사이즈 후 uint8 텐서로 변환하여 리스트에 저장
    """
    to_tensor = transforms.PILToTensor() # uint8 유지 (0-255)
    resizer = ResizeWithPad(224)

    tensor_list = []
    print(f"Loading {len(paths)} images to RAM (uint8 mode)...")

    for path in tqdm(paths):
        try:
            img = Image.open(path).convert('RGB')
            img = resizer(img)
            tensor = to_tensor(img) # (3, 224, 224) uint8 Tensor
            tensor_list.append(tensor)
        except Exception as e:
            print(f"Error loading {path}: {e}")


    return torch.stack(tensor_list) # (N, 3, 224, 224) uint8 Tensor

In [8]:
class AlexNetLike(nn.Module):
    def __init__(self, num_classes=1):
        super(AlexNetLike, self).__init__()
        self.features = nn.Sequential(
            # Conv 1
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2), # 224 -> 55
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), # 55 -> 27

            # Conv 2
            nn.Conv2d(96, 256, kernel_size=5, padding=2), # 27 -> 27
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), # 27 -> 13

            # Conv 3
            nn.Conv2d(256, 384, kernel_size=3, padding=1), # 13 -> 13
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            # Conv 4
            nn.Conv2d(384, 384, kernel_size=3, padding=1), # 13 -> 13
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            # Conv 5
            nn.Conv2d(384, 256, kernel_size=3, padding=1), # 13 -> 13
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), # 13 -> 6
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes) # num_classes=1
        )
    def forward(self, x):
        x = self.features(x); x = self.avgpool(x)
        x = torch.flatten(x, 1); x = self.classifier(x)
        return x

In [9]:
def get_model(model_name, device):

    model = None
    num_classes = 1 # 이진 분류 (Real/Fake)

    print(f"Loading {model_name} architecture (FROM SCRATCH)...")

    if model_name.lower() == 'alexnet':
        # 직접 짠 AlexNet (Conv 5, FC 3)
        model =  AlexNetLike(num_classes=num_classes)

    elif model_name.lower() == 'vgg16':
        # VGG16
        model = models.vgg16(weights=None, num_classes=num_classes)

    elif model_name.lower() == 'googlenet':
        # GoogLeNet
        model = models.googlenet(weights=None, num_classes=num_classes, aux_logits=False)

    elif model_name.lower() == 'resnet50':
        # ResNet50
        model = models.resnet50(weights=None, num_classes=num_classes)

    else:
        raise ValueError(f"Unknown model name: {model_name}. Choose from 'alexnet, 'vgg16', 'googlenet', 'resnet50'")

    return model.to(device)

In [10]:
# 학습
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs, patience):
    print("=== 학습 시작 ===")

    best_val_loss = float('inf')
    best_model_weights = None
    epochs_no_improve = 0

    for epoch in range(epochs):
        start_time = time.time()

        # --- 훈련 ---
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # tqdm으로 진행 상황 표시
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)

        for images, labels in train_pbar:
            images, labels = images.to(device), labels.to(device)

            # 순전파
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            # 정확도 계산
            preds = torch.sigmoid(outputs) > 0.5
            total_train += labels.size(0)
            correct_train += (preds == labels).sum().item()

            train_pbar.set_postfix({'loss': loss.item()})

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct_train / total_train

        # --- 검증 ---
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item() * images.size(0)

                preds = torch.sigmoid(outputs) > 0.5
                total_val += labels.size(0)
                correct_val += (preds == labels).sum().item()

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc = correct_val / total_val

        elapsed_time = time.time() - start_time

        print(f"Epoch {epoch+1}/{epochs} - {elapsed_time:.0f}s - "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f} - "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

        # --- Early Stopping 및 Best Model 저장 ---
        if epoch_val_loss < best_val_loss:
            print(f"  Validation loss decreased ({best_val_loss:.4f} --> {epoch_val_loss:.4f}). Saving model...")
            best_val_loss = epoch_val_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            torch.save(best_model_weights, MODEL_SAVE_PATH)
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f"  Validation loss did not improve. Patience: {epochs_no_improve}/{patience}")

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    print("=== 학습 완료 ===")
    model.load_state_dict(best_model_weights)
    return model

In [11]:
# 평가
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    print("\n=== 테스트셋 평가 시작 ===")
    with torch.no_grad():
        test_pbar = tqdm(test_loader, desc="[Test]", leave=False)
        for images, labels in test_pbar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_test_loss += loss.item() * images.size(0)

            preds = torch.sigmoid(outputs) > 0.5
            total_test += labels.size(0)
            correct_test += (preds == labels).sum().item()

    test_loss = running_test_loss / len(test_loader.dataset)
    test_acc = correct_test / total_test

    print(f"===== 최종 테스트 결과 =====")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
# 메인 실행
start_time = time.time()

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
torch.backends.cudnn.benchmark = True

# 데이터 경로 및 라벨 수집
Fake_PATH = "C:/Users/user/Downloads/Dataset/Dataset/Train/Fake"
Real_PATH = "C:/Users/user/Downloads/Dataset/Dataset/Train/Real"
MODEL_SAVE_PATH = "./ML/deepfake_baseline_model.pth"
print("이미지 경로 수집 중...")
face_real_dir = os.path.join(Real_PATH)
face_fake_dir = os.path.join(Fake_PATH)

real_paths = glob.glob(os.path.join(face_real_dir, "*.*"))
fake_paths = glob.glob(os.path.join(face_fake_dir, "*.*"))

all_paths = real_paths + fake_paths
all_labels = [0] * len(real_paths) + [1] * len(fake_paths)

print(f"총 {len(all_labels)}개 이미지 경로 발견.")
# 샘플 개수 제한
NUM_SAMPLES = min(NUM_SAMPLES, len(all_paths))

print(f"{NUM_SAMPLES}개 샘플을 샘플링...")
_, target_paths, _, target_labels = train_test_split(
    all_paths, all_labels,
    test_size= NUM_SAMPLES,
    random_state=42,
    stratify=all_labels
)
print(f"샘플링 완료: {NUM_SAMPLES}개 이미지 선택.")
target_paths = all_paths
target_labels = all_labels
    
# 전체를 쓰더라도 학습을 위해 섞어줍니다.
combined = list(zip(target_paths, target_labels))
np.random.shuffle(combined)
target_paths[:], target_labels[:] = zip(*combined)

Using device: cuda
이미지 경로 수집 중...
총 140002개 이미지 경로 발견.
20000개 샘플을 샘플링...
샘플링 완료: 20000개 이미지 선택.


In [16]:
print("데이터를 7:2:1 비율로 분할합니다...")
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    target_paths, target_labels, test_size=0.3, random_state=42, stratify=target_labels
)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=(1/3), random_state=42, stratify=temp_labels
)
print(f"분할 완료: Train {len(train_paths)}개, Validation {len(val_paths)}개, Test {len(test_paths)}개")

데이터를 7:2:1 비율로 분할합니다...
분할 완료: Train 14000개, Validation 4000개, Test 2000개


In [17]:
# 데이터 전처리 및 RAM에 로드
print("데이터 전처리를 시작합니다 (모든 데이터를 RAM에 로드)...")

# 모든 데이터를 RAM으로 로드
X_train = load_images_to_ram_uint8(train_paths)
y_train = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1)

X_val = load_images_to_ram_uint8(val_paths)
y_val = torch.tensor(val_labels, dtype=torch.float32).unsqueeze(1)

X_test = load_images_to_ram_uint8(test_paths)
y_test = torch.tensor(test_labels, dtype=torch.float32).unsqueeze(1)

print("모든 데이터를 RAM에 로드 완료.")

# RAM 기반의 TensorDataset과 DataLoader 생성
train_dataset = RAMDataset(X_train, y_train, transform=None)
val_dataset = RAMDataset(X_val, y_val, transform=None)
test_dataset = RAMDataset(X_test, y_test, transform=None)

# RAM에서 읽으므로 num_workers=0, pin_memory=False (이미 RAM에 있음)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print("데이터 로더 생성 완료.")

데이터 전처리를 시작합니다 (모든 데이터를 RAM에 로드)...
Loading 14000 images to RAM (uint8 mode)...


100%|██████████| 14000/14000 [04:26<00:00, 52.50it/s]


Loading 4000 images to RAM (uint8 mode)...


100%|██████████| 4000/4000 [01:18<00:00, 50.85it/s]


Loading 2000 images to RAM (uint8 mode)...


100%|██████████| 2000/2000 [00:37<00:00, 53.52it/s]


모든 데이터를 RAM에 로드 완료.
데이터 로더 생성 완료.


In [18]:
start_time = time.time()
# 8. 모델, 손실함수, 옵티마이저 정의
model = get_model('alexnet', device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 9. 학습 및 평가
model = train_model(model, train_loader, val_loader, criterion, optimizer, device, EPOCHS, PATIENCE)
evaluate_model(model, test_loader, criterion, device)

print(f"총 실행 시간: {(time.time() - start_time) / 60:.2f} 분")

Loading alexnet architecture (FROM SCRATCH)...
=== 학습 시작 ===


Epoch 1/30 - 703s - Train Loss: 0.7071, Train Acc: 0.5821 - Val Loss: 0.6171, Val Acc: 0.6783
  Validation loss decreased (inf --> 0.6171). Saving model...


RuntimeError: Parent directory ./ML does not exist.